In [78]:
import pandas

train_data = pandas.read_csv('dataset/Kaggle_Training_Dataset.csv')
train_data = train_data.set_index('sku')

In [79]:

test_data = pandas.read_csv('dataset/Kaggle_Test_Dataset.csv')
test_data = test_data.set_index('sku')

In [80]:
# print(train_data[:10])

In [81]:
#print all the columns/features
print("shape =",train_data.shape)
print("columns = ", train_data.columns.values)

shape = (1693050, 22)
columns =  ['national_inv' 'lead_time' 'in_transit_qty' 'forecast_3_month'
 'forecast_6_month' 'forecast_9_month' 'sales_1_month' 'sales_3_month'
 'sales_6_month' 'sales_9_month' 'min_bank' 'potential_issue'
 'pieces_past_due' 'perf_6_month_avg' 'perf_12_month_avg' 'local_bo_qty'
 'deck_risk' 'oe_constraint' 'ppap_risk' 'stop_auto_buy' 'rev_stop'
 'went_on_backorder']


In [82]:
#distribution of 'went_on_backorder'
yes_shape = train_data[train_data['went_on_backorder'] == 'Yes'].shape
no_shape = train_data[train_data['went_on_backorder'] == 'No'].shape
print("went_on_backorder = Yes = ", yes_shape)
print("went_on_backorder = No = ", no_shape)
print("went_on_backorder = % of Yes/no = ", yes_shape[0]*100.0/train_data.shape[0])


went_on_backorder = Yes =  (10914, 22)
went_on_backorder = No =  (1682136, 22)
went_on_backorder = % of Yes/no =  0.6446354212811198


In [83]:
#not_went_on_backorder = train_data[train_data['went_on_backorder'] == 'No']
#went_on_backorder = train_data[train_data['went_on_backorder'] == 'Yes']
#import matplotlib.pyplot as plt


columns = ['potential_issue','deck_risk','oe_constraint','ppap_risk','stop_auto_buy','rev_stop']

for col in columns:
    print(col)
    print("Went on backorder")
    yes_len = train_data[(train_data[col] == 'Yes') & (train_data['went_on_backorder'] == 'Yes')].shape[0]
    #no_len = train_data[(train_data[col] == 'No') & (train_data['went_on_backorder'] == 'Yes')].shape[0]
    print(yes_len);
    #print(no_len);
    print("YES % = ",yes_len*100.0/(train_data.shape[0]))
    #print("NO % = ",no_len*100.0/(yes_len+no_len))
    print("============================")
    

potential_issue
Went on backorder
35
YES % =  0.0020672750361773132
deck_risk
Went on backorder
1577
YES % =  0.09314550663004637
oe_constraint
Went on backorder
5
YES % =  0.0002953250051681876
ppap_risk
Went on backorder
1578
YES % =  0.09320457163108001
stop_auto_buy
Went on backorder
10434
YES % =  0.6162842207849739
rev_stop
Went on backorder
17
YES % =  0.0010041050175718378


In [84]:
#dig deeper into stop_auto_buy

print ("Total size = ", train_data.shape[0])
print ("True positive = ",train_data[(train_data['stop_auto_buy'] == 'Yes') & (train_data['went_on_backorder'] == 'Yes')].shape[0])
print ("True negative = ",train_data[(train_data['stop_auto_buy'] == 'No') & (train_data['went_on_backorder'] == 'No')].shape[0])
print ("False positive = ",train_data[(train_data['stop_auto_buy'] == 'Yes') & (train_data['went_on_backorder'] == 'No')].shape[0])
print ("False negative = ",train_data[(train_data['stop_auto_buy'] == 'No') & (train_data['went_on_backorder'] == 'Yes')].shape[0])

Total size =  1693050
True positive =  10434
True negative =  65233
False positive =  1616903
False negative =  480


In [85]:
def process(df):
    """We turn some of the features in to binary."""
    for col in ['potential_issue', 'deck_risk', 'oe_constraint', 'ppap_risk',
               'stop_auto_buy', 'rev_stop', 'went_on_backorder']:
        df[col] = (df[col] == 'Yes').astype(int)
    return df
train_data = process(train_data).dropna()
test_data = process(test_data).dropna()
from sklearn.ensemble import RandomForestClassifier

est = RandomForestClassifier(n_jobs=-1, n_estimators=100, min_samples_split=10,
                             max_depth=10, class_weight='balanced')
X, y = train_data.drop('went_on_backorder', axis=1), train_data.went_on_backorder

In [86]:
est.fit(X, y)
X2, y2 = test_data.drop('went_on_backorder', axis=1), test_data.went_on_backorder
preds = est.predict_proba(X2)

In [88]:
import numpy as np
from sklearn.metrics import classification_report
pred_labels = np.argmax(preds, axis=1)
print(classification_report(y2, pred_labels))

             precision    recall  f1-score   support

          0       1.00      0.89      0.94    225210
          1       0.07      0.82      0.13      2280

avg / total       0.99      0.89      0.93    227490

